In [1]:
## IMPORTAMOS LIBRERIAS ##
from cmath import nan
import pandas as pd
from pandas.io import gbq
from datetime import date, timedelta, datetime
import pandas_gbq
import ctypes
import chardet
import os
import win32com.client

## DEFINIMOS FECHAS DE CARGA ##
today =         date.today()
load_date =     today - timedelta(days = 1)
date_loaded =   load_date.strftime("%d/%m/%Y")
load_day =      int(load_date.strftime("%d"))
load_month =    load_date.strftime("%m")
load_year =     load_date.strftime("%Y")

## DEFINIMOS LISTA CON MESES PARA IR A LA UBICACION EXACTA Y CARGAMOS EL DIRECTORIO DE FILES ##
meses = ['01. ENERO', '02. FEBRERO','03. MARZO','04. ABRIL','05. MAYO','06. JUNIO','07. JULIO','08. AGOSTO','09. SEPTIEMBRE','10. OCTUBRE','11. NOVIEMBRE','12. DICIEMBRE']
path = fr"v:\Central reservations office\Area-3\SOPORTE OPERACIONES\\01. INFORMES DIARIOS\\{load_year}\\{meses[int(load_month)-1]} {load_year}\\{load_day}"

In [ ]:
# Conectarse a Outlook
outlook = win32com.client.Dispatch("Outlook.Application").GetNamespace("MAPI")
inbox = outlook.Folders.Item("b.intelligence@minor-hotels.com").Folders.Item("Bandeja de entrada")
messages = inbox.Items
messages.Sort("[ReceivedTime]", True)  # Ordenar por fecha de recepción descendente

# Procesar los primeros 5 mensajes
for i in range(min(5, messages.Count)):
    message = messages.Item(i + 1)  
    attachments = message.Attachments
    if message.Subject.startswith('CRO_Cr'):
        attachments.Item(1).SaveAsFile(path + "\\CRO_TMS_Reservations.xlsx")
        message.Delete()
    elif message.Subject.startswith('CRO_Le'):
        attachments.Item(1).SaveAsFile(path + "\\CRO_LeadTime.xlsx")
        message.Delete()


In [ ]:
## DEFINIMOS LISTA CON MESES PARA IR A LA UBICACION EXACTA Y CARGAMOS EL DIRECTORIO DE FILES ##
files = os.listdir(fr"G:\Central reservations office\Area-3\SOPORTE OPERACIONES\\01. INFORMES DIARIOS\\{load_year}\\{meses[int(load_month)-1]} {load_year}\\{load_day}")

## PROCEDEMOS A LA CARGA DE ARCHIVOS A CLOUD CON UN BUCLE QUE RECORRE LOS ARCHIVOS DEL DIRECTORIO PARA ENCONTRAR LOS 2 A SUBIR ##
for file in files: 

    # BUSCAMOS EL ARCHIVO CRO_VS_HOTEL EN CLOUD PARA SABER SI TIENE DATOS Y NO DUPLICAR. SI SE HA SUBIDO, PASAMOS AL SIGUIENTE. SI NO SE HA SUBIDO, PROCEDEMOS A SUBIRLO ##
    if file[:6]=='CRO_Le':
        CHECK_CRO_VS_HOTEL = pandas_gbq.read_gbq("""SELECT Creation_date_origin FROM `nh-cro-forecast.TMS.c_Reservation_Share_Hotel_vs_CRO` ORDER BY Creation_date_origin DESC LIMIT 1""",project_id="nh-cro-forecast")

        if int(pd.to_datetime(CHECK_CRO_VS_HOTEL.values[0][0]).strftime("%d"))==int(load_day):
            print('CRO_VS_HOTEL DUPLICADO')
        else:
            file1 = fr"G:\Central reservations office\Area-3\SOPORTE OPERACIONES\\01. INFORMES DIARIOS\\{load_year}\\{meses[int(load_month)-1]} {load_year}\\{load_day}\\{file}"
            CRO_vs_Hotel = pd.read_excel(file1).astype(str).replace(['NaN','None','NaT',' ','nan',nan],[None,None,None,None,None,None])
            CRO_vs_Hotel.columns = ['Hotel_ID','Hotel_name','Hotel_business_unit','Hotel_country','Hotel_city','Creation_year_month','Department','RO_channel','RO_subchannel_ID','Reservation_origin_ID','Total_revenue','Total_room_revenue',
                                    'Total_fb_revenue','Total_meeting_room_revenue','Total_other_revenue','Room_nights','Creation_date_origin','OK_LT']
            CRO_vs_Hotel["Creation_date_origin"] = pd.to_datetime(CRO_vs_Hotel["Creation_date_origin"]).dt.strftime('%Y-%m-%d')
            CRO_vs_Hotel["Rooms"] = '1'
            CRO_vs_Hotel['Reservation_origin_ID'] = CRO_vs_Hotel['Reservation_origin_ID'].astype(float).astype(int).astype(str)
            print('CRO_VS_HOTEL SUBIDO')
            CRO_vs_Hotel.to_gbq(destination_table='TMS.c_Reservation_Share_Hotel_vs_CRO',project_id='nh-cro-forecast',if_exists='append')

    # BUSCAMOS EL ARCHIVO TMS EN CLOUD PARA SABER SI TIENE DATOS Y NO DUPLICAR. SI SE HA SUBIDO, PASAMOS AL SIGUIENTE. SI NO SE HA SUBIDO, PROCEDEMOS A SUBIRLO ##
    else:
        if file[:6]=='CRO_TM':
            CHECK_TMS = pandas_gbq.read_gbq("""select RS_FECHA_RESERVA from `nh-cro-forecast.TMS.a_Reservation_entry_data_2022_raw` ORDER BY RS_FECHA_RESERVA DESC LIMIT 1""",project_id="nh-cro-forecast")

            if int(pd.to_datetime(CHECK_TMS.values[0][0]).strftime("%d"))==int(load_day):
                print('TMS DUPLICADO')
            else:
                file2 = fr"G:\Central reservations office\Area-3\SOPORTE OPERACIONES\\01. INFORMES DIARIOS\\{load_year}\\{meses[int(load_month)-1]} {load_year}\\{load_day}\\{file}"
                TMS = pd.read_excel(file2,dtype=str).replace(['NaN','None','NaT',' ','nan',nan],[None,None,None,None,None,None]).rename(columns={'Hotel Currency':'Hotel_Currency',
                                                                                                                                                    'Status Reservation Origin ID':'Status_Reservation_Origin_ID',
                                                                                                                                                    'RO Room type generic (upgrade)':'RO_Room_type_generic_upgrade',
                                                                                                                                                    'RO Commercial Subchannel':'RO_Commercial_Subchannel',
                                                                                                                                                    'RO Reservation Done By':'RO_Reservation_Done_by',
                                                                                                                                                    'RO Contact person':'RO_Contact_person',
                                                                                                                                                    'Room Nights':'Room_Nights',
                                                                                                                                                    'Room Nights CXL':'Room_Nights_Cancellation',
                                                                                                                                                    'Room Nights No Show':'Room_Nights_No_Show',
                                                                                                                                                    'Room Nights OK':'Room_Nights_Ok',
                                                                                                                                                    'Total Book Revenue FIN EUR':'Total_Book_Revenue_FIN_EUR',
                                                                                                                                                    'Book Room Revenue EUR':'Book_Room_Revenue_EUR',
                                                                                                                                                    'Book Breakfast Revenue EUR':'Book_Breakfast_Revenue_EUR',
                                                                                                                                                    'Cancelled Book Revenue EUR':'Cancelled_Book_Revenue_EUR',
                                                                                                                                                    'No Show Book Revenue EUR':'No_Show_Book_Revenue_EUR',
                                                                                                                                                    'Total Other Revenue FIN EUR (new)':'Total_Other_Revenue_FIN_EUR_new',
                                                                                                                                                    'Total Book Revenue FIN LC':'Total_Book_Revenue_FIN_LC',
                                                                                                                                                    'Book Room Revenue LC':'Book_Room_Revenue_LC',
                                                                                                                                                    'Book Breakfast Revenue LC':'Book_Breakfast_Revenue_LC',
                                                                                                                                                    'Cancelled Book Revenue LC':'Cancelled_Book_Revenue_LC',
                                                                                                                                                    'No Show Book Revenue LC':'No_Show_Book_Revenue_LC',
                                                                                                                                                    'Total Other Revenue FIN LC (new)':'Total_Other_Revenue_FIN_LC_new',
                                                                                                                                                    'RO Reservation type':'RO_Reservation_type',
                                                                                                                                                    'RO Branch Company Responsible TMS':'RO_Branch_Company_Responsible_TMS',
                                                                                                                                                    'RO Guarantee Category':'RO_Guarantee_Category',
                                                                                                                                                    'Creation Time Origin':'Creation_Time_Origin',
                                                                                                                                                    'Creation Date Origin CET':'Creation_Date_Origin_CET',
                                                                                                                                                    'Creation Time Origin CET':'Creation_Time_Origin_CET'})
                TMS['RS_RESERVA'] = TMS['RS_RESERVA'].astype(float).astype(int)
                TMS = TMS.astype(str)
                TMS.replace(['NaN','None','NaT',' ','nan',nan],[None,None,None,None,None,None], inplace=True)
                print('TMS SUBIDO')
                TMS.to_gbq(destination_table='TMS.b_Reservation_entry_data_raw_temporal',project_id='nh-cro-forecast',if_exists='replace')
                TMS.to_gbq(destination_table='TMS.a_Reservation_entry_data_2022_raw',project_id='nh-cro-forecast',if_exists='append')
        else:
            next